In [46]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from math import floor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import pandas as pd
from random import sample

#standardize actual fp

In [47]:
def reduceForProjectionAnalysis(file):
    file = file[file['injury_status'] != 'O']
    file = file[file['injury_status'] != 'Q']
    file['FantasyFuelPPGProj'] = file['ppg_projection']
    file['FantasyFuelValueProj'] = file['value_projection']
    file['DFN_PPGProj'] = file['Proj FP']
    file['DFN_ValueProj'] = file['Proj Val']
    
    cleaned_file = pd.DataFrame()
    cleaned_file['Player Name'] = file['Player Name']
    cleaned_file['Actual FP'] = file['Actual FP']
    cleaned_file['team'] = file['team']
    cleaned_file['Pos'] = file['Pos']
    cleaned_file['Salary'] = file['Salary']
    cleaned_file['DFN_MinProj'] = file['Proj Min']
    cleaned_file['Avg Proj'] = (file['DFN_PPGProj']+file['FantasyFuelPPGProj'])/2
    cleaned_file['Avg Value Proj'] = ((file['DFN_ValueProj']+file['FantasyFuelValueProj'])/2)
    cleaned_file['Avg Skewed Min'] = ((file['L2 Min']+file['L5 Min']+file['S Min'])/3)
    cleaned_file['Avg Skewed FP'] = ((file['L5 FP']+file['S FP']+file['Ceil FP']+file['Floor FP']+cleaned_file['Avg Proj'])/5)
    cleaned_file['Proj Min Enhanced'] = (cleaned_file['Avg Proj'] / cleaned_file['Avg Skewed Min'] ) * cleaned_file['DFN_MinProj']
    cleaned_file = cleaned_file.replace([np.inf, -np.inf], 0)
    cleaned_file = cleaned_file.fillna(0)
    return cleaned_file

def standardize(df):
    result = df.copy()
    categories = ['Avg Proj', 'DFN_MinProj', 'Proj Min Enhanced', 'Avg Skewed FP', 'Avg Value Proj', 'Actual FP']
    for feature_name in categories:
        mean_value = df[feature_name].mean()
        std_value = df[feature_name].std()
        result[feature_name + ' Stan'] = ((df[feature_name] - mean_value) / (std_value))
    result = result.replace([np.inf, -np.inf], 0)
    result = result.fillna(0)
    return result

def buildRegressionModel(master_training_data, cleaned_file):
    drop_attribs = ['Player Name','team','Pos','Salary', 'Actual FP', 'Actual FP Stan', 'Avg Proj', 'DFN_MinProj', 'Proj Min Enhanced', 'Avg Skewed FP', 'Avg Skewed Min', 'Avg Value Proj']
    features = master_training_data.drop(drop_attribs, axis=1).copy()
    labels = master_training_data['Actual FP Stan'].copy()
    reg = LinearRegression()
    reg.fit(features, labels)
    scores = cross_val_score( reg, features, labels, scoring='neg_mean_squared_error', cv=10)
    rmse_scores = np.sqrt(-scores)
    display_scores(rmse_scores, 'Actual FP')
    cleaned_file['predictions'] = reg.predict(cleaned_file.drop(drop_attribs, axis=1).copy())
    return cleaned_file
    
    
def display_scores(scores, label_type):
    print("*", label_type)
    print('Scores:', scores)
    print('Mean:' ,scores.mean())
    print('Std:', scores.std())  
    
    
    

In [48]:
##Helper Functions
def verifyLineup(lineup):
    enough_represented_teams = len(set(lineup['team'].tolist()))>=2
    under_salary_cap = lineup['Salary'].sum() <= 50000
    all_unique_players = len(set(lineup['Player Name'].tolist())) == 8
    if enough_represented_teams and under_salary_cap and all_unique_players:
        return True
    else:
        return False
    
def createRandomPopulation(point_guards, shooting_guards, small_forwards, power_forwards, guards, forwards, centers, util, limit):
    counter = 0
    lineups = []
    while(counter<limit):
        lineup = pd.DataFrame()
        lineup = lineup.append(point_guards.sample(n=1))
        lineup = lineup.append(shooting_guards.sample(n=1))
        lineup = lineup.append(small_forwards.sample(n=1))
        lineup = lineup.append(power_forwards.sample(n=1))
        lineup = lineup.append(guards.sample(n=1))
        lineup = lineup.append(forwards.sample(n=1))
        lineup = lineup.append(centers.sample(n=1))
        lineup = lineup.append(util.sample(n=1))
        if(verifyLineup(lineup)):
            lineups.append(lineup)
            counter = counter + 1
    return lineups

def mate(_parents):
    parents = pd.concat(sample(_parents, 10))
    point_guards = parents.loc[parents['Pos'].str.contains("PG")]
    shooting_guards = parents.loc[parents['Pos'].str.contains("SG")]
    small_forwards = parents.loc[parents['Pos'].str.contains("SF")]
    power_forwards = parents.loc[parents['Pos'].str.contains("PF")]
    guards = parents.loc[parents['Pos'].str.contains("G")]
    forwards = parents.loc[parents['Pos'].str.contains("F")]
    centers = parents.loc[parents['Pos'].str.contains("C")]
    util = parents
    children = []
    while(len(children) < 10):
        child_lineup = _parents[0].append(_parents[1])
        while(not verifyLineup(child_lineup)):
            child_lineup = pd.DataFrame()
            child_lineup = child_lineup.append(point_guards.sample(n=1))
            child_lineup = child_lineup.append(shooting_guards.sample(n=1))
            child_lineup = child_lineup.append(small_forwards.sample(n=1))
            child_lineup = child_lineup.append(power_forwards.sample(n=1))
            child_lineup = child_lineup.append(guards.sample(n=1))
            child_lineup = child_lineup.append(forwards.sample(n=1))
            child_lineup = child_lineup.append(centers.sample(n=1))
            child_lineup = child_lineup.append(util.sample(n=1))
        children.append(child_lineup)
    return children

def sortFitestBasedOnProjection(population): 
    population.sort(key=projectedPointSum, reverse=True)
    population = eliminateDuplicates(population)
    return population 

def projectedPointSum(lineup):
    return lineup['Avg Proj'].sum()

def eliminateDuplicates(population):
    unique_lineups = set()
    unique_population = []
    for lineup in population:
        lineup_set = set()
        for player in lineup['Player Name'].tolist():
            lineup_set.add(player) 
        if(not lineup_set.issubset(unique_lineups)):  
            unique_population.append(lineup)
            unique_lineups = unique_lineups.union(lineup_set)
    return unique_population


In [49]:
def performSelection(population):
    selected_population = population[:1]
    selected_population.extend(sample(population[1:5], 3))
    selected_population.extend(sample(population[5:10], 2))
    selected_population.extend(sample(population, 10))
    return selected_population


def performCrossover(population):
    children = []
    parents = population
    children = mate(parents) 
    return children
        

def createNextGeneration(old_population):
    selected_population = performSelection(old_population)
    children = performCrossover(selected_population)
    new_generation = old_population[:len(old_population)-10]
    new_generation.extend(children)
    return new_generation
    
        

In [50]:
def geneticAlgorithmForHistorialBestLineups(file):
    population_size = 30
    point_guards = file.loc[file['Pos'].str.contains("PG")]
    shooting_guards = file.loc[file['Pos'].str.contains("SG")]
    small_forwards = file.loc[file['Pos'].str.contains("SF")]
    power_forwards = file.loc[file['Pos'].str.contains("PF")]
    guards = file.loc[file['Pos'].str.contains("G")]
    forwards = file.loc[file['Pos'].str.contains("F")]
    centers = file.loc[file['Pos'].str.contains("C")]
    util = file
    population = createRandomPopulation(point_guards, shooting_guards, 
                                               small_forwards, power_forwards, 
                                               guards, forwards, centers, util, population_size)  
   
    counter = 0 
    while(counter < 1600 ):
        currentPopulation = sortFitestBasedOnProjection(population)
        population = createNextGeneration(currentPopulation)
        population.extend(createRandomPopulation(point_guards, shooting_guards, 
                                               small_forwards, power_forwards, 
                                               guards, forwards, centers, util, 30 - len(population)))
        counter = counter+1
       
    print("*****************")
    return sortFitestBasedOnProjection(population)

In [51]:
dates_all = [
    ['3_11', '2020-03-11'],
    ['3_10', '2020-03-10'],
    ['3_9', '2020-03-09'],
    ['3_8', '2020-03-08'],
    ['3_7', '2020-03-07'],
    ['3_6', '2020-03-06'],
    ['3_5', '2020-03-05'],
    ['3_4', '2020-03-04'],
    ['3_3', '2020-03-03'],
    ['3_2', '2020-03-02'],
    ['3_1', '2020-03-01'],
    ['2_29', '2020-02-29'],
    ['2_28', '2020-02-28'],
    ['2_27', '2020-02-27'],
    ['2_26', '2020-02-26'],
    ['2_25', '2020-02-25'],
    ['2_24', '2020-02-24'],
    ['2_23', '2020-02-23'],
    ['2_22', '2020-02-22'],
    ['2_21', '2020-02-21'],
    ['2_20', '2020-02-20'],
    ['2_13', '2020-02-13'],
    ['2_12', '2020-02-12'],
    ['2_11', '2020-02-11'],
    ['2_10', '2020-02-10'],
    ['2_9', '2020-02-09'],
    ['2_8', '2020-02-08'],
    ['2_7', '2020-02-07'],
    ['2_6', '2020-02-06'],
    ['2_5', '2020-02-05'],
    ['2_4', '2020-02-04'],
    ['2_3', '2020-02-03'],
    ['2_2', '2020-02-02'],
    ['2_1', '2020-02-01'],
    ['1_31', '2020-01-31'],
    ['1_30', '2020-01-30'],
    ['1_29', '2020-01-29'],
    ['1_28', '2020-01-28'],
    ['1_27', '2020-01-27'],
    ['1_26', '2020-01-26'],
    ['1_25', '2020-01-25'],
    ['1_24', '2020-01-24'],
    ['1_23', '2020-01-23'],
    ['1_22', '2020-01-22'],
    ['1_20', '2020-01-20'],
    ['1_19', '2020-01-19'],
    ['1_18', '2020-01-18'],
    ['1_17', '2020-01-17'],
    ['1_16', '2020-01-16'],
    ['1_15', '2020-01-15'],
    ['1_14', '2020-01-14'],
    ['1_13', '2020-01-13'],
    ['1_12', '2020-01-12'],
    ['1_11', '2020-01-11'],
    ['1_10', '2020-01-10'],
    ['1_9', '2020-01-09'],
    ['1_8', '2020-01-08'],
    ['1_7', '2020-01-07'],
    ['1_6', '2020-01-06'],
    ['1_5', '2020-01-05'],
    ['1_4', '2020-01-04'],
    ['1_3', '2020-01-03'],
    ['1_2', '2020-01-02'],
    ['1_1', '2020-01-01'],
    ['12_31', '2019-12-31'],
    ['12_30', '2019-12-30'],
    ['12_29', '2019-12-29'],
    ['12_28', '2019-12-28'],
    ['12_27', '2019-12-27'],
    ['12_26', '2019-12-26'],
    ['12_25', '2019-12-25'],
    ['12_23', '2019-12-23'],
    ['12_22', '2019-12-22'],
    ['12_21', '2019-12-21'],
    ['12_20', '2019-12-20'],
    ['12_19', '2019-12-19'],
    ['12_18', '2019-12-18'],
    ['12_17', '2019-12-17'],
    ['12_16', '2019-12-16'],
    ['12_15', '2019-12-15'],
    ['12_14', '2019-12-14'],
    ['12_13', '2019-12-13'],
    ['12_12', '2019-12-12'],
    ['12_11', '2019-12-11'],
    ['12_10', '2019-12-10'],
    ['12_9', '2019-12-09'],
    ['12_8', '2019-12-08'],
    ['12_7', '2019-12-07'],
    ['12_6', '2019-12-06'],
    ['12_5', '2019-12-05'],
    ['12_4', '2019-12-04'],
    ['12_3', '2019-12-03'],
    ['12_2', '2019-12-02'],
    ['12_1', '2019-12-01'],
    ['11_30', '2019-11-30'],
    ['11_29', '2019-11-29'],
    ['11_27', '2019-11-27'],
    ['11_26', '2019-11-26'],
    ['11_25', '2019-11-25'],
    ['11_24', '2019-11-24'],
    ['11_23', '2019-11-23'],
    ['11_22', '2019-11-22'],
    ['11_21', '2019-11-21'],
    ['11_20', '2019-11-20'],
    ['11_19', '2019-11-19'],
    ['11_18', '2019-11-18'],
    ['11_17', '2019-11-17'],
    ['11_16', '2019-11-16'],
    ['11_15', '2019-11-15'],
    ['11_14', '2019-11-14'],
    ['11_13', '2019-11-13'],
    ['11_12', '2019-11-12'],
    ['11_11', '2019-11-11'],
    ['11_10', '2019-11-10'],
    ['11_9', '2019-11-09'],
    ['11_8', '2019-11-08'],
    ['11_7', '2019-11-07'],
    ['11_6', '2019-11-06'],
    ['11_5', '2019-11-05'],
    ['11_4', '2019-11-04'],
    ['11_3', '2019-11-03'],
    ['11_2', '2019-11-02'],
    ['11_1', '2019-11-01'],
    ['10_31', '2019-10-31'],
    ['10_30', '2019-10-30'],
    ['10_29', '2019-10-29'],
    ['10_28', '2019-10-28'],
    ['10_27', '2019-10-27'],
    ['10_26', '2019-10-26'],
    ['10_25', '2019-10-25'],
    ['10_24', '2019-10-24'],
    ['10_23', '2019-10-23'],
    ['10_22', '2019-10-22'],   
]


def resetIndicies(file):
    for lineup in file:
        lineup.reset_index(drop=True, inplace=True)
    return file

def saveBestLineups_toCSV(file, date):
    key = np.arange(len(file))
    file_name = 'BestCreatedLineups/{}'.format(date)
    new_file = pd.concat(file, keys=key, names=['Lineup Num'])
    new_file.to_csv(file_name)

def createBestLineups():
    build_limit = 0
    master_training_data = pd.DataFrame()
    dates_all.reverse()
    for game_day in dates_all:
        print(game_day[1])
        file_name = 'HistoricalData_Merged/{}'.format(game_day[1])
        file = pd.read_csv(file_name)
        cleaned_file = standardize(reduceForProjectionAnalysis(file))
        if(build_limit > 3):
            file_with_predictions = buildRegressionModel(master_training_data, cleaned_file)
            master_training_data = master_training_data.append(cleaned_file.drop(['predictions'], axis=1), sort=False)
            best_population = geneticAlgorithmForHistorialBestLineups(file_with_predictions)
            saveBestLineups_toCSV(resetIndicies(best_population), game_day[1])
        else:
            master_training_data = master_training_data.append(cleaned_file, sort=False)
        build_limit = build_limit + 1
        

In [52]:
createBestLineups()



2019-10-22
2019-10-23
2019-10-24
2019-10-25
2019-10-26
* Actual FP
Scores: [0.76160052 0.79161115 0.5993087  0.56877387 0.56294337 0.62016639
 0.72641421 0.70626732 0.58622402 0.43606616]
Mean: 0.6359375711493187
Std: 0.10344431976791342
*****************
2019-10-27
* Actual FP
Scores: [0.82420872 0.61703174 0.6095354  0.58821087 0.6667631  0.67829015
 0.50315501 0.83607954 0.61426687 0.47220035]
Mean: 0.6409741749939821
Std: 0.1123548668626921
*****************
2019-10-28
* Actual FP
Scores: [0.80156464 0.61004653 0.59886596 0.59523441 0.73798127 0.56732812
 0.73908382 0.67794477 0.48257223 0.55811259]
Mean: 0.6368734346551524
Std: 0.09392898789694033
*****************
2019-10-29
* Actual FP
Scores: [0.77685293 0.58557007 0.57572801 0.72134764 0.5381376  0.79835329
 0.47874923 0.58571139 0.7155885  0.6994031 ]
Mean: 0.6475441746032947
Std: 0.10271983629159047
*****************
2019-10-30
* Actual FP
Scores: [0.75821035 0.61545871 0.55120578 0.71356726 0.6266629  0.69758996
 0.55444666

*****************
2019-12-05
* Actual FP
Scores: [0.67334597 0.66539679 0.65429456 0.68854405 0.61226114 0.64398615
 0.66581881 0.66001371 0.6183492  0.63335718]
Mean: 0.6515367571981218
Std: 0.02313486530885437
*****************
2019-12-06
* Actual FP
Scores: [0.6739741  0.66291561 0.6558819  0.68166747 0.62335208 0.64944004
 0.65675482 0.65423552 0.61484311 0.64530026]
Mean: 0.6518364919756273
Std: 0.01943542114078972
*****************
2019-12-07
* Actual FP
Scores: [0.6700715  0.65897671 0.65334955 0.67551812 0.63513655 0.65701871
 0.65574196 0.65327539 0.61742488 0.62326202]
Mean: 0.6499775372943287
Std: 0.01798826422887166
*****************
2019-12-08
* Actual FP
Scores: [0.66706042 0.65976136 0.66167015 0.66687742 0.64316828 0.64465934
 0.66839174 0.63701865 0.62255345 0.634987  ]
Mean: 0.6506147821994632
Std: 0.015376190785560696
*****************
2019-12-09
* Actual FP
Scores: [0.66448884 0.65458131 0.68044675 0.64972132 0.63734744 0.65841222
 0.66139813 0.63809246 0.63893609 0

*****************
2020-01-14
* Actual FP
Scores: [0.67177953 0.66910454 0.63101979 0.66045377 0.63725855 0.62995829
 0.64598607 0.63271274 0.65342783 0.64231745]
Mean: 0.647401856278385
Std: 0.014802381618375516
*****************
2020-01-15
* Actual FP
Scores: [0.67199417 0.66357905 0.63652584 0.66281235 0.63158885 0.64047572
 0.63320342 0.63149609 0.65245286 0.65417412]
Mean: 0.6478302457184253
Std: 0.014298275724141477
*****************
2020-01-16
* Actual FP
Scores: [0.6704811  0.67176486 0.63311983 0.66556977 0.62249726 0.63440378
 0.64508233 0.62844457 0.64471376 0.66011862]
Mean: 0.6476195894901673
Std: 0.017270415132024822
*****************
2020-01-17
* Actual FP
Scores: [0.66934802 0.67600689 0.62919899 0.66819172 0.62741103 0.62552697
 0.64239545 0.63517052 0.64659086 0.65192162]
Mean: 0.6471762068161322
Std: 0.01771111410634238
*****************
2020-01-18
* Actual FP
Scores: [0.66865221 0.67838833 0.63110549 0.66287931 0.63057422 0.62250294
 0.64157656 0.63881252 0.65380887 

*****************
2020-02-29
* Actual FP
Scores: [0.66883003 0.64566678 0.65840749 0.62654986 0.6378689  0.65065583
 0.64776021 0.63213425 0.61505659 0.6167915 ]
Mean: 0.6399721427229003
Std: 0.016685336102734528
*****************
2020-03-01
* Actual FP
Scores: [0.66860459 0.64379645 0.65626373 0.63198341 0.63671042 0.64733917
 0.65538249 0.62974952 0.61179234 0.61324269]
Mean: 0.639486481578541
Std: 0.01754074445094009
*****************
2020-03-02
* Actual FP
Scores: [0.66853104 0.64580337 0.65188595 0.63694757 0.63056695 0.64645541
 0.65615811 0.6350117  0.60568124 0.61360678]
Mean: 0.6390648122800343
Std: 0.018112285220263206
*****************
2020-03-03
* Actual FP
Scores: [0.66838199 0.64804903 0.65153223 0.63376    0.62816374 0.64908072
 0.65850366 0.62750545 0.60731882 0.62104519]
Mean: 0.6393340830616971
Std: 0.01784998575424279
*****************
2020-03-04
* Actual FP
Scores: [0.66819824 0.64801281 0.65352327 0.62691856 0.63010407 0.65351193
 0.6523813  0.63353689 0.59904321 0